In [2]:
import pandas as pd

df = pd.read_csv("DemandPrediction_Final.csv", header=None)

In [3]:
#df.iloc[:,df.columns != 2]

# df.drop([2,5,8,11], axis=1)

df = df[[0,1,4,7,10,13]]

In [4]:
new_header = df.iloc[0]
df = df[1:] 
new_header[0] = 'Date'
df.columns = new_header


In [5]:
SEQ_LEN = 60  # how long of a preceeding sequence to collect for RNN
FUTURE_PERIOD_PREDICT = 3  # how far into the future are we trying to predict?
main_df = df

In [6]:
RATIO_TO_PREDICT = "LOS ANGLES"
main_df[f'{RATIO_TO_PREDICT}_future'] = df[RATIO_TO_PREDICT].shift(-FUTURE_PERIOD_PREDICT)

In [7]:
main_df.head()

,Date,LOS ANGLES,ATLANTA,WASHANGTON,NEW YORK,SEATEL,LOS ANGLES_future
1,2015/1/1,11284,65145,2196,19452,5136,14660
2,2015/1/2,15418,94584,3093,25512,7273,13206
3,2015/1/3,13835,83493,2477,22200,7190,12545
4,2015/1/4,14660,94283,2909,24565,7347,13052
5,2015/1/5,13206,98617,3553,23419,5985,13328


In [8]:
def classify(current, future):
    if float(future) > float(current):
        return 1
    else:
        return 0

In [9]:
RATIO_TO_PREDICT = "LOS ANGLES"

main_df[f'{RATIO_TO_PREDICT}_target'] = list(map(classify, main_df[RATIO_TO_PREDICT], main_df[f'{RATIO_TO_PREDICT}_future']))

In [10]:
main_df.head()

,Date,LOS ANGLES,ATLANTA,WASHANGTON,NEW YORK,SEATEL,LOS ANGLES_future,LOS ANGLES_target
1,2015/1/1,11284,65145,2196,19452,5136,14660,1
2,2015/1/2,15418,94584,3093,25512,7273,13206,0
3,2015/1/3,13835,83493,2477,22200,7190,12545,0
4,2015/1/4,14660,94283,2909,24565,7347,13052,0
5,2015/1/5,13206,98617,3553,23419,5985,13328,1


In [11]:
main_df.Date= pd.to_datetime(main_df.Date)
main_df = main_df.set_index('Date')


In [12]:
main_df.head()

,LOS ANGLES,ATLANTA,WASHANGTON,NEW YORK,SEATEL,LOS ANGLES_future,LOS ANGLES_target
Date,,,,,,,
2015-01-01,11284,65145,2196,19452,5136,14660,1
2015-01-02,15418,94584,3093,25512,7273,13206,0
2015-01-03,13835,83493,2477,22200,7190,12545,0
2015-01-04,14660,94283,2909,24565,7347,13052,0
2015-01-05,13206,98617,3553,23419,5985,13328,1


In [34]:
times = sorted(main_df.index.values)  # get the times
last_5_Percentage = sorted(main_df.index.values)[-int(0.05*len(times))]  # get the last 5% of the times

validation_main_df = main_df[(main_df.index >= last_5_Percentage)]  # make the validation data where the index is in the last 5%
main_df = main_df[(main_df.index < last_5_Percentage)]  # now the main_df is all the data up to the last 5%

In [18]:
from sklearn import preprocessing
from collections import deque
import numpy as np
import random

def preprocess_df(df):
    df = df.drop(columns=[f'{RATIO_TO_PREDICT}_future'])  # don't need this anymore.

    for col in df.columns:  # go through all of the columns
        if col != f'{RATIO_TO_PREDICT}_target':  # normalize all ... except for the target itself!
            df[col] = df[col].astype(float).pct_change()  
            # pct change "normalizes" the different currencies (each crypto coin has vastly diff values, we're really more interested in the other coin's movements)
            df.dropna(inplace=True)  # remove the nas created by pct_change
            df[col] = preprocessing.scale(df[col].values)  # scale between 0 and 1.

    df.dropna(inplace=True)  # cleanup again... jic.


    sequential_data = []  # this is a list that will CONTAIN the sequences
    prev_days = deque(maxlen=SEQ_LEN)  # These will be our actual sequences. They are made with deque, which keeps the maximum length by popping out older values as new ones come in

    for i in df.values:  # iterate over the values
        prev_days.append([n for n in i[:-1]])  # store all but the target
        if len(prev_days) == SEQ_LEN:  # make sure we have 60 sequences!
            sequential_data.append([np.array(prev_days), i[-1]]) # append those bad boys!

    random.shuffle(sequential_data)  # shuffle for good measure.
    buys = []  # list that will store our buy sequences and targets
    sells = []  # list that will store our sell sequences and targets

    for seq, target in sequential_data:  # iterate over the sequential data
        if target == 0:  # if it's a "not buy"
            sells.append([seq, target])  # append to sells list
        elif target == 1:  # otherwise if the target is a 1...
            buys.append([seq, target])  # it's a buy!

    random.shuffle(buys)  # shuffle the buys
    random.shuffle(sells)  # shuffle the sells!

    lower = min(len(buys), len(sells))  # what's the shorter length?

    buys = buys[:lower]  # make sure both lists are only up to the shortest length.
    sells = sells[:lower]  # make sure both lists are only up to the shortest length.

    sequential_data = buys+sells  # add them together
    random.shuffle(sequential_data)  # another shuffle, so the model doesn't get confused with all 1 class then the other.

    X = []
    y = []

    for seq, target in sequential_data:  # going over our new sequential data
        X.append(seq)  # X is the sequences
        y.append(target)  # y is the targets/labels (buys vs sell/notbuy)

    return np.array(X), y  # return X and y...and make X a numpy array!

In [35]:
train_x, train_y = preprocess_df(main_df)
validation_x, validation_y = preprocess_df(validation_main_df)

In [36]:
validation_main_df

,LOS ANGLES,ATLANTA,WASHANGTON,NEW YORK,SEATEL,LOS ANGLES_future,LOS ANGLES_target
Date,,,,,,,
2015-02-01,11346,79146,2902,17664,4791,13472,1
2015-02-02,13260,102806,3808,24042,5971,13401,1
2015-02-03,12972,93852,4101,21318,5756,13405,1
2015-02-04,13472,98426,4247,21356,5502,11435,0
2015-02-05,13401,102485,4097,23814,5789,13146,0
2015-02-06,13405,102394,4097,23675,5963,13016,0
2015-02-07,11435,75457,2383,15619,5392,12968,1
2015-02-08,13146,95657,3519,21583,5829,13364,1
2015-02-09,13016,102914,3804,23556,6018,13163,1


In [31]:
print(f"train data: {len(train_x)} validation: {len(validation_x)}")
print(f"Dont buys: {train_y.count(0)}, buys: {train_y.count(1)}")
print(f"VALIDATION Dont buys: {validation_y.count(0)}, buys: {validation_y.count(1)}")

train data: 0 validation: 42
Dont buys: 0, buys: 0
VALIDATION Dont buys: 21, buys: 21


## <div style="direction:rtl;text-align:right;font-family:B Lotus, B Nazanin, Tahoma">منبع:</div>

https://becominghuman.ai/recurrent-neural-networks-rnn-deep-learning-w-python-tensorflow-keras-p-7-c21bc374d4dc

Cryptocurrency-predicting RNN Model

In [22]:
import time

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization

EPOCHS = 10  # how many passes through our data
BATCH_SIZE = 64  # how many batches? Try smaller batch if you're getting OOM (out of memory) errors.


In [23]:
model = Sequential()
model.add(LSTM(128, input_shape=(train_x.shape[1:]), return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())  #normalizes activation outputs, same reason you want to normalize your input data.

model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.1))
model.add(BatchNormalization()) # optional

model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(BatchNormalization()) # optional

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(2, activation='softmax'))  # softmax gets OneHot 
# During these 60 time steps they will not return output except the last one
# ------------------v---------------------
#model.add(Dense(2, activation='softmax'))
# OR
#model.add(Dense(1, activation='sigmoid'))

In [25]:
opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)

# Compile model
model.compile(
    loss='sparse_categorical_crossentropy', # instead of OneHot multi-labled class, it will chenge labale to OneHot
    optimizer=opt,
    metrics=['accuracy']
)

In [26]:
# Train model
history = model.fit(
    train_x, np.array(train_y),
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(validation_x, np.array(validation_y)))

Epoch 1/10
2/2 [==============================] - 2s 985ms/step - loss: 0.9568 - accuracy: 0.5119 - val_loss: 0.6922 - val_accuracy: 0.5909
Epoch 2/10
2/2 [==============================] - 0s 211ms/step - loss: 0.8072 - accuracy: 0.6071 - val_loss: 0.6920 - val_accuracy: 0.5568
Epoch 3/10
2/2 [==============================] - 0s 207ms/step - loss: 0.7279 - accuracy: 0.7024 - val_loss: 0.6924 - val_accuracy: 0.5000
Epoch 4/10
2/2 [==============================] - 0s 224ms/step - loss: 0.5222 - accuracy: 0.6905 - val_loss: 0.6925 - val_accuracy: 0.5000
Epoch 5/10
2/2 [==============================] - 0s 207ms/step - loss: 0.5188 - accuracy: 0.6905 - val_loss: 0.6925 - val_accuracy: 0.5000
Epoch 6/10
2/2 [==============================] - 0s 215ms/step - loss: 0.5902 - accuracy: 0.7857 - val_loss: 0.6927 - val_accuracy: 0.5000
Epoch 7/10
2/2 [==============================] - 0s 208ms/step - loss: 0.4128 - accuracy: 0.8214 - val_loss: 0.6928 - val_accuracy: 0.4545
Epoch 8/10
2/2 [====

In [27]:
# Score model
score = model.evaluate(validation_x, np.array(validation_y), verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.6928997039794922
Test accuracy: 0.5113636255264282
